In [22]:
import os
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras
import keras.backend as K
from keras import layers, Model
from keras.optimizers import Adam


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf

from modules.dataset import DatasetFactory
from modules.encoding import NLFEncoder

from modules.data.loader import SequenceEncodingDataGenerator

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
dataset_dir = "../data/SAbDab"

df_match = pd.read_csv(os.path.join(dataset_dir, "data.csv"), sep=";")
df_seq = pd.read_csv(os.path.join(dataset_dir, "sequences.csv"), sep=";")

In [25]:
df_match.head()

,ab,ag,interaction
0,5kel|ab,5kel|ag,1
1,5kel|ab,6cwt|ag,0
2,5kel|ab,4fp8|ag,0
3,5kel|ab,4yjz|ag,0
4,5kel|ab,6j15|ag,0


### Filter 

Remove keys which does not have both an antigen and an anticorps.

In [26]:
df_filtered = df_match[df_match["ab"].isin(df_seq["seq_id"]) & df_match["ag"].isin(df_seq["seq_id"])]
df_filtered.shape

(345323, 3)

In [27]:
df_filtered.to_csv(os.path.join(dataset_dir, "data_filtered.csv"), sep=";", index=False)

In [28]:
df_match.shape

(421821, 3)

## Encoding

### NLF

We want to encode the sequence first, and retrieve the encoded for each sequence in the join later.

In [29]:
encoder = NLFEncoder()

In [30]:
seq = df_seq["sequence"].values
encoded_seq = encoder.encode(seq, vector_size=2000)

In [31]:
seq_id_to_nlf = {seq_id: nlf for seq_id, nlf in zip(df_seq["seq_id"], encoded_seq)}

In [35]:
X = df_filtered[["ab", "ag"]] 
y = df_filtered[["interaction"]]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [36]:
train_ds_loader = SequenceEncodingDataGenerator(x_train, y_train, seq_id_to_nlf, batch_size=32)
test_ds_loader = SequenceEncodingDataGenerator(x_test, y_test, seq_id_to_nlf, batch_size=32)

In [37]:
example_batch = train_ds_loader[0]
input_dimensions = example_batch[0][0].shape
input_dimensions

(2000, 18)

## Siamese network

In [38]:
seq_input1 = layers.Input(shape=input_dimensions, name='seq_ag')
seq_input2 = layers.Input(shape=input_dimensions, name='seq_ab')

In [39]:
# Convolutional modules
filters = 96


conv01 = layers.Conv1D(filters, 11, padding='same', activation="relu")
mp1 = layers.MaxPooling1D(3)
conv02 = layers.Conv1D(filters*2, 7, padding='same', activation="relu")
mp2 = layers.MaxPooling1D(3)
conv03 = layers.Conv1D(filters*4, 3, padding='same', activation="relu")
mp3 = layers.MaxPooling1D(3)
conv04 = layers.Conv1D(filters*2, 3, padding='same', activation="relu")
mp4 = layers.MaxPooling1D(3)

gru = layers.Bidirectional(layers.GRU(filters, return_sequences=False))

def siamese_propagation(x):
    x = conv01(x)
    x = mp1(x)

    x = conv02(x)
    x = mp2(x)

    x = conv03(x)
    x = mp3(x)

    x = conv04(x)
    x = mp4(x)

    x_gru = gru(x)
    return x_gru



In [40]:
def forward(left, right):
    left = siamese_propagation(left)
    right = siamese_propagation(right)

    merge = layers.multiply([left, right])

    # merge = layers.Dense(filters*2, activation='relu')(merge)
    merge = layers.Dropout(0.2)(merge)
    return layers.Dense(1, activation='sigmoid')(merge)

In [41]:
def f1(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    pred_pos = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = tp / (pred_pos + K.epsilon())
    recall = tp / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def mcc(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return numerator / (denominator + K.epsilon())

def accuracy(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
    return (tp + tn) / (tp + tn + fp + fn)

In [42]:
def binary_crossentropy(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    loss = - K.mean(y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))
    return loss

In [43]:
subset = train_ds_loader[:10]


In [44]:
subset_ag = []
subset_ab = []
subset_y = []

for item in subset:
    subset_x = item[0]
    ag, ab = subset_x
    for x in [ag, ab]:
        x = x.reshape((-1, 2000, 18))
        x = x[0]
    subset_ag.append(ag)
    subset_ab.append(ab)
    subset_y.append(item[1])

subset_ag = np.array(subset_ag)
subset_ab = np.array(subset_ab)
subset_y = np.array(subset_y)
subset_ag.shape

(10, 2000, 18)

In [45]:
model = Model(inputs=[seq_input1, seq_input2],
            outputs=[forward(seq_input1, seq_input2)])

adam = Adam(learning_rate=1e-4, amsgrad=True, epsilon=1e-6)

checkpoint_callback = keras.callbacks.ModelCheckpoint('run/siamese/model.h5', monitor='val_mcc', mode='max')
earlystop_callback = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

model.compile(optimizer=adam, loss=binary_crossentropy, metrics=[accuracy, f1, mcc])


model.fit([subset_ag, subset_ab], subset_y, epochs=50, callbacks=[checkpoint_callback, earlystop_callback],
          batch_size=64, verbose=1, validation_split=0.15)


Epoch 1/50
1/1 [==============================] - 8s 8s/step - loss: 0.6747 - accuracy: 1.0000 - f1: 0.0000e+00 - mcc: 0.0000e+00 - val_loss: 0.6645 - val_accuracy: 1.0000 - val_f1: 0.0000e+00 - val_mcc: 0.0000e+00
Epoch 2/50
1/1 [==============================] - ETA: 0s - loss: 0.6652 - accuracy: 1.0000 - f1: 0.0000e+00 - mcc: 0.0000e+00

/home/sortion/.local/share/miniforge3/envs/intelligent-antibodies/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 227ms/step - loss: 0.6652 - accuracy: 1.0000 - f1: 0.0000e+00 - mcc: 0.0000e+00 - val_loss: 0.6494 - val_accuracy: 1.0000 - val_f1: 0.0000e+00 - val_mcc: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 231ms/step - loss: 0.6492 - accuracy: 1.0000 - f1: 0.0000e+00 - mcc: 0.0000e+00 - val_loss: 0.6306 - val_accuracy: 1.0000 - val_f1: 0.0000e+00 - val_mcc: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 265ms/step - loss: 0.6311 - accuracy: 1.0000 - f1: 0.0000e+00 - mcc: 0.0000e+00 - val_loss: 0.6074 - val_accuracy: 1.0000 - val_f1: 0.0000e+00 - val_mcc: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 213ms/step - loss: 0.6032 - accuracy: 1.0000 - f1: 0.0000e+00 - mcc: 0.0000e+00 - val_loss: 0.5794 - val_accuracy: 1.0000 - val_f1: 0.0000e+00 - val_mcc: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 199ms/step - loss: 0.5783 - accuracy: 1.0000 - f1: 0.0000e+00 - mcc: 0.0000e+00 - val_

In [ ]:
len(train_ds_loader)

276258

In [ ]:
seq_input1

<KerasTensor: shape=(None, 2, 2000, 18) dtype=float32 (created by layer 'seq_ag')>

In [ ]:
# Siamese model using GRU
seq_input1 = layers.Input(shape=input_dimensions, name='seq_ag')
seq_input2 = layers.Input(shape=input_dimensions, name='seq_ab')

# Convolutional modules


In [ ]:
df_filtered.head()

,ab,ag,interaction
0,5kel|ab,5kel|ag,1
1,5kel|ab,6cwt|ag,0
2,5kel|ab,4fp8|ag,0
3,5kel|ab,4yjz|ag,0
4,5kel|ab,6j15|ag,0


In [ ]:
df_filtered["ab"].dtype == 